# transient-dynamics Colab
## Description
This Colab notebook contains a pipeline that uses weights based on observational principles to investigate the causes of dynamical shifts in the Lotka-Volterra predator-prey models applied to a phage-bacteria systems. The outputs include Excel sheets for the inputted case scenario (solved with an ODE solver) and a case solved through an adaptive model using observational principles through a boolean lens.

The notebook also provides figures highlighting the weight of each term: growth, predation, burst, and decay. It shows the population in both full and adaptive models, processes relevant in terms of observational principles, and a relative error percentage analysis between the two models.




# Input structure and parameters
## Input Structure
'transient-dynamics' requires some base inputs to define the user's observational window and the parameters defining the phage bacteria system. The list below describes the varibales and their options:
+ 'tobs': The observational time of the experiment, how long are you investigating the system. This would impact the total time but not the time steps created.
+ 'reference_level': The perception factor of interest to the observer, the default used within the paper (Cobo-Lopez et al,2025) is 0.10 (10% change).
+ 'r': Bacterial growth rate in terms of units (1/hour); quantifies the proportional change per hour of the bacteria.
+ 'a': Adsorption rate in terms of units (ml/hour); quantifies the probaility per unit time that a single phage particle will bind to a suspectible host cell.
+ 'c': Burst rate of infected cell given in terms of (# infectious particles/lytic cycle); characterizes the number of phage particles released in a single infectious/lytic cycle
+ 'm': Decay rate of phage in terms of propotional loss of phage per hour.
+ 'bi': Initial bacterial concentration inside the experiment.
+ 'pi': Initial phage concentration inside the experiment.
+ 'k': Carrying capacity of bacteria in the experiment: it is given in terms of bacterial concentration (cells/ml).

## Input parameters
Input panel in which you can choose the parameters to run the experiment of the phage-bacteria. For the ease of use, the cases used within the paper (Cobo-Lopez et al., 2025) are listed within this panel for demonstratory purposes.
### Scenario inputs:


**Scenario 1** (Laboratory case analog E.coli-like bacteria & T4-like phage): r = 9.0 * 10**-1 ; a = 3.0 * 10**-8 ; c = 50 ; m = 1.5 * 10**-3 ; tobs = 14 ; bi = 1 * 10** 3 ; pi = 1 * 10** 4 ; reference_level = 0.1 ; thresholdlevel = 1.0 &nbsp;


**Scenario 2** (Environmental UV-exposed marine-like case of slow growing bacteria and fast decaying phage): r = 3.2 * 10**-5 ; a = 1.0 * 10**-10 ; c = 225 ; tobs = 260 ; bi = 6.67 * 10** 7 ; pi = 1 * 10**0 ; reference_level = 0.1 ; thresholdlevel = 1.0 &nbsp;


**Scenario 3 Transient** (Environmental enriched coastal like case of relatively fast-growing bacteria and fast decaying phage, displaying transient dynamics): r = 1.0 * 10**-1 ; a = 3.0 * 10**-9 ; c = 50 ; m = 1.0 * 10**-1 ; tobs = 300 ; bi = 2.65  * 10 **3 ; pi = 4.93 * 10 **4 &nbsp;


**Scenario 3 Quasi-equilibrium** (Environmental enriched coastal like case of relatively fast-growing bacteria and fast decaying phage, displaying quasi-equilibrium): r = 1.0 * 10**-1 ; a = 3.0 * 10**-9 ; c = 50 ; m = 1.0 * 10**-1 ; tobs = 300 ; bi = 6.0  * 10 **5 ; pi = 6.0 * 10 **6 &nbsp;


**Scenario 4 Transient** (Laboratory with deep ocean-like case of slow growing bacteria and slow decaying phage, displaying transient dynamics): r = 3.2 * 10**-5 ; a = 3.0 * 10**-8 ; c = 50 ; m = 1.5 * 10**-3 ; tobs = 13 ; bi = 8.5 * 10 **5 ; pi = 3.0 * 10 **3 &nbsp;


**Scenario 4 Quasi-static** (Laboratory with deep ocean-like case of slow growing bacteria and slow decaying phage, displaying transient dynamics): r = 3.2 * 10**-5 ; a = 3.0 * 10**-8 ; c = 50 ; m = 1.5 * 10**-3 ; tobs = 13 ; bi = 1.0 * 10 **3 ; pi = 1.0 * 10 **4 &nbsp;

In [1]:
# Growth rate parameter of bacteria (1/hr)
r = 0.9

# Adsorption rate, the rate at which phages can enter a bacteria (ml/hr)
a = 3.0 * 10**-8

# Burst size, the amount of viral particles generated by the bursting of a bacteria cell
c = 50

# Decay rate of phage
m = 1.5 * 10**-3

# Observational time, the elapsed time during the simulation (hr)
tobs = 14

# Initial bacterial population (cells/ml)
bi = 10**3

# Initial phage population (virions/ml)
pi = 10**4

# The alpha level, or the amount of change that is perceptible by the viewer 
reference_level = 0.10

# The weight value above which the processes will be considered relevant, should be kept at one
thresholdlevel = 1.0

# Store the parameters in a tuple for testing or further use
para_test = (r, a, c, m, tobs, reference_level, thresholdlevel)

# Import packages
## These imports are required to handle operations utilized in code:


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint, solve_ivp
import statistics as stat

## These imports are requird to handle the visualization of the plots:

In [3]:
import openpyxl
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
from scipy import stats
from matplotlib.ticker import MaxNLocator

# Functions and storage lists
## Functions used for simulation and figure generation of standard Lotka-Volterra model

In [4]:
# Used to calculate the weights for each given process based upon set weight function
def weights_finder(weight_function, time_steps, weight_store):
  '''
  This function is used to calculate the weights for each given process based on a set weight function.

  Parameters:
  - weight_function: A list or array containing the values of the weight function for each time step.
  - time_steps: A list or array representing the time steps.
  - weight_store: An empty list to store the calculated weights.

  Returns:
  - weight_store: A list containing the calculated weights for each time step.
  '''
  weight_store = []
  for i in range(len(time_steps)):
    weight_store_step = weight_function[i]
    weight_store.append(weight_store_step)
  return weight_store
    
# Used to find the tipping points based upon the switching of weights above threshold which is then collected.
def tipping_point_finder(weight_list, time_steps, tipping_point_list, threshholdlevel):
  '''
  This function is used to find the tipping points based on the switching of weights above or below a threshold level.

  Parameters:
  - weights_list: A list or array containing the values of the weights at each time step.
  - time_steps: A list or array representing the time steps.
  - tipping_point_list: An empty list to store the calculated tipping points.
  - threshholdlevel: The threshold level used to determine the tipping points.

  Returns:
  - tipping_point_list: A list containing the calculated tipping points.
  '''
  tipping_point_list = []
  for i in range(len(time_steps[1:])):
    if weight_list[i] >= threshholdlevel and weight_list[i-1] < threshholdlevel:
      if time_steps[i] != 0:
        tipping_point_list.append(time_steps[i])
    elif weight_list[i] < threshholdlevel and weight_list[i-1] >= threshholdlevel:
      if time_steps[i] != 0:
        tipping_point_list.append(time_steps[i])
    else:
      continue
  return tipping_point_list
    
# Counts the number of relevant processes based upon values of the weights, threshold and point in time.
def process_counter(Thresholdlevel, weight_list_1, weight_list_2, weight_list_3, weight_list_4, time_steps, relevant_process_list):
  '''
  This function is used to count the number of relevant processes based on the weights and a given threshold level.

  Parameters:
  - Thresholdlevel: The threshold level used to determine the relevance of the processes.
  - weight_list_1, weight_list_2, weight_list_3, weight_list_4: Lists or arrays containing the values of the weights at each time step for each process.
  - time_steps: A list or array representing the time steps.
  - relevant_process_list: An empty list to store the calculated number of relevant processes.

  Returns:
  - relevant_process_list: A list containing the number of relevant processes at each time step.
  '''
  relevant_process_list = []
  for i in range(len(time_steps)):
    if weight_list_1[i] < Thresholdlevel:
      process_1 = 0
    else:
      process_1 = 1
    if weight_list_2[i] < Thresholdlevel:
      process_2 = 0
    else:
      process_2 = 1
    if weight_list_3[i] < Thresholdlevel:
      process_3 = 0
    else:
      process_3 = 1
    if weight_list_4[i] < Thresholdlevel:
      process_4 = 0
    else:
      process_4 = 1
    process_index = process_1 + process_2 + process_3 + process_4
    relevant_process_list.append(process_index)
  return relevant_process_list

# Function to calculate the absolute error between the adaptive model and the full model, this is used on both agents.
def absolute_error(time_steps, agent_full_model, agent_adaptive_model, error_store_term):
  '''
  This function is used to calculate the absolute error between the adaptive model and the full model for both agents.

  Parameters:
  - time_steps: A list or array representing the time steps.
  - agent_full_model: A list or array containing the values of the full model for the agents at each time step.
  - agent_adaptive_model: A list or array containing the values of the adaptive model for the agents at each time step.
  - error_store_term: An empty list to store the calculated absolute error between the adaptive model and the full model.

  Returns:
  - error_store_term: A list containing the absolute error between the adaptive model and the full model at each time step.
  '''
  error_store_term = []
  for i in range(len(time_steps)):
    error_store_term_i = abs(agent_adaptive_model[i] - agent_full_model[i])
    error_store_term.append(error_store_term_i)
  return error_store_term

# Function that calculates the relative error through dividing the absolute error by the full model value at each time step.
def relative_error(time_steps, agent_absolute_error, agent_full, relative_error):
  '''
  This function calculates the relative error by dividing the absolute error by the full model value at each given unit of time.

  Parameters:
  - time_steps: A list or array representing the time steps.
  - agent_absolute_error: A list containing the absolute error between the adaptive model and the full model at each time step.
  - agent_full: A list or array containing the values of the full model for the agents at each time step.
  - relative_error: An empty list to store the calculated relative error.

  Returns:
  - relative_error: A list containing the relative error at each time step.
  '''
  relative_error = []
  for i in range(len(time_steps)):
    relative_error_input = (agent_absolute_error[i] / agent_full[i])
    relative_error.append(relative_error_input)
  return relative_error

# Take the relative error and convert it to percentage (multiplies by 100).
def relative_error_percentage_finder(relative_error, time_steps, error_percentage):
  '''
  This function takes the relative error and converts it to a percentage by multiplying it by 100.

  Parameters:
  - time_steps: A list or array representing the time steps.
  - relative_error: A list containing the relative error at each time step.
  - error_percentage: An empty list to store the calculated relative error as a percentage.

  Returns:
  - error_percentage: A list containing the relative error as a percentage at each time step.
  '''
  error_percentage = []
  for i in range(len(time_steps)):
    error_percentage_input = relative_error[i] * 100
    error_percentage.append(error_percentage_input)
  return error_percentage

  # This graph creates a line graph of the dynamics of both agents represented in both models, additionally, any tipping points will appear on the graph with the line pointing to the critical value that create a tipping point.
  def population_plot_comparison_and_tipping_points(time_steps, time_final, sum_tipping_point_list, predation_tipping_points, burst_tipping_points, phage_full_model, bacteria_full_model, phage_adaptive_model, bacteria_adaptive_model):
    """
    Plot the population comparison and tipping points of both models.

    Parameters:
    - time_steps (list): List of time values.
    - time_final (float): Final time value.
    - sum_tipping_point_list (list): List of all observed tipping points.
    - predation_tipping_points (list): List of tipping points for predation process.
    - burst_tipping_points (list): List of tipping points for burst process.
    - phage_full_model (list): Population of phage in the full model.
    - bacteria_full_model (list): Population of bacteria in the full model.
    - phage_adaptive_model (list): Population of phage in the adaptive model.
    - bacteria_adaptive_model (list): Population of bacteria in the adaptive model.

    Returns:
    - None
    """
    print('Below are the graphs representing both the tipping points and populations plot of both models')

    # Calculate the figure size to maintain the 227:127 ratio
    width_inches = 3.632
    height_inches = width_inches * (114 / 215)

    # Set up the figure and subplots
    fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=250)

    ax.semilogy(time_steps, bacteria_full_model, label='Bacteria', color='b', linestyle='solid', linewidth=0.75)
    ax.semilogy(time_steps, phage_full_model, label='Phage', color='r', linestyle='solid', linewidth=0.75)
    ax.semilogy(time_steps, bacteria_adaptive_model, label='Adaptive', color='k', linestyle='dashed', dashes=(5,5), linewidth=0.75)
    ax.semilogy(time_steps, phage_adaptive_model, color='k', linestyle='dashed', dashes=(5,5), linewidth=0.75)

    ax.set_xlabel("Time (h)", fontsize=7, labelpad=2)
    ax.set_ylabel('Concentration ($ml^{-1}$)', fontsize=7, labelpad=2)
    ax.set_xlim(0, time_final)

    if round(np.log10(max(phage_full_model))) > round(np.log10(max(bacteria_full_model))):
      max_limit = (1 * (10**round(np.log10(max(phage_full_model)))))
      ax.set_ylim(1, max_limit)
    else:
      max_limit = (1 * (10**round(np.log10(max(bacteria_full_model)))))
      ax.set_ylim(1, max_limit)

    if max_limit < 10**9:
      ax.set_yticks([10**0, 10**3, 10**6, 10**9])
    else:
      ax.set_yticks([10**0, 10**3, 10**6, 10**9, 10**12])

    # Format y-axis labels as 10^x
    def y_fmt(y, pos):
      decades = int(np.log10(y))
      return r'$10^{%d}$' % decades

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))

    ax.minorticks_off()

    # Create evenly distributed tick marks for x-axis
    num_ticks = 6  # You can adjust this number as needed
    x_ticks = np.linspace(0, time_final, num_ticks)
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([f'{x:.0f}' for x in x_ticks])

    # Setting up the vertical lines for tipping points
    if len(sum_tipping_point_list) != 0:
      for tps in sum_tipping_point_list:
        if tps in predation_tipping_points:
          ax.vlines(x=tps, ymin=0, ymax=(float(phage_full_model[np.where(time_steps == tps)[0][0]])), color='tab:grey', linestyle='solid', linewidth=0.75)
        elif tps in burst_tipping_points:
          ax.vlines(x=tps, ymin=0, ymax=(float(bacteria_full_model[np.where(time_steps == tps)[0][0]])), color='tab:grey', linestyle='solid', linewidth=0.75)
        else:
          print('error')

    ax.legend(fontsize=5, loc='best')

    # Adjust tick parameters
    ax.tick_params(axis='both', which='major', labelsize=6, pad=2)

    plt.tight_layout(pad=0.1)

    # Add a thin border around the entire figure
    fig.patch.set_linewidth(0.5)
    fig.patch.set_edgecolor('black')

    # Save and show the figure
    plt.savefig('population_comparison_graph.svg', dpi=250, bbox_inches='tight')
    plt.savefig('population_comparison_graph.eps', dpi=250, bbox_inches='tight')
    plt.savefig('population_comparison_graph.png', dpi=250, bbox_inches='tight')
    plt.show()

# This function plots the weights as line graphs (subplots for each fo the weights), with the selected threshold level forming a line to distinguish where the weights would be relevant and irrelevant. The tipping point were shown as a vertical line.
def plot_weights_graphs(time_final, threshholdlevel, reference_level, weight_growth, weight_predation, weight_burst, weight_decay, predation_tipping_points, burst_tipping_points):
  """
  Plots the weights as line graphs with the selected threshold level and tipping points.

  Parameters:
  - time_final (float): The final time value.
  - threshholdlevel (float): The threshold level for the weights.
  - reference_level (float): The reference level for the plot settings.
  - weight_growth (array-like): The growth weight values.
  - weight_predation (array-like): The predation weight values.
  - weight_burst (array-like): The burst weight values.
  - weight_decay (array-like): The decay weight values.
  - predation_tipping_points (array-like): The tipping points for predation weight.
  - burst_tipping_points (array-like): The tipping points for burst weight.

  Returns:
  None
  """
  # Create time array
  t = np.linspace(0, time_final, len(weight_growth))

  # Set up the figure and subplots
  fig, axweight = plt.subplots(4, 1, sharex=True, figsize=(3.632, 1.824), dpi=250)

  # Define plot settings based on reference_level
  if reference_level == 1:
    y_lim = (1e-4, 1e4)
    y_ticks = [1e-3, 1, 1e3]
  elif reference_level == 0.10:
    y_lim = (1e-5, 1e5)
    y_ticks = [1e-4, 1, 1e4]
  else:
    raise ValueError("Unsupported reference_level")

  # Plot data
  plot_data = [
    (weight_growth, 'Growth Weight', 'Growth', 'b'),
    (weight_predation, 'Predation Weight', 'Predation', 'b'),
    (weight_burst, 'Burst Weight', 'Burst', 'r'),
    (weight_decay, 'Decay Weight', 'Decay', 'r')
  ]

  # Calculate the margin to add (e.g., 5% of the total time range)
  margin = time_final * 0

  for i, (weight, label, ylabel, color) in enumerate(plot_data):
    ax = axweight[i]
    ax.semilogy(t, weight, label=label, color=color, linewidth=1.50)
    ax.yaxis.set_label_position("right")
    ax.set_ylabel(ylabel, fontsize=5, labelpad=2)
    ax.set_ylim(y_lim)
    ax.set_yticks(y_ticks)
    ax.axhline(y=threshholdlevel, color='tab:grey', linestyle='dotted', linewidth=0.75)
    ax.axhline(y=0, color='k', linewidth=0.75)
    ax.axhspan(1e-5, 1e0, facecolor='tab:grey', alpha=0.3)

    # Format y-axis labels as 10^x
    def y_fmt(y, pos):
      if y == 1:
        return '1'
      decades = int(np.log10(y))
      return r'$10^{%d}$' % decades

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))

    # Adjust tick parameters
    ax.tick_params(axis='both', which='major', labelsize=6, pad=2, width=0.9)
    ax.tick_params(axis='both', which='minor', width=0.6)
    if i < 3:  # Remove x-axis labels for all but the last subplot
      ax.tick_params(axis='x', labelbottom=False)
    if i == 1:
      for tps in predation_tipping_points:
        ax.vlines(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)
    if i == 2:
      for tps in burst_tipping_points:
        ax.vlines(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)

    # Keep all spines visible
    for spine in ax.spines.values():
      spine.set_visible(True)
      spine.set_linewidth(1.15)

  # Set x-label for the bottom subplot
  axweight[-1].set_xlabel('Time (h)', fontsize=7, labelpad=2)

  # Set x-axis scale based on time_final
  if time_final > 1000:
    for ax in axweight:
      ax.set_xscale('log')
    x_ticks = [10**i for i in range(int(np.log10(time_final))+1)]
    axweight[-1].set_xticks(x_ticks)
    axweight[-1].set_xticklabels([f'$10^{int(np.log10(x))}$' if x > 1 else '1' for x in x_ticks])
    axweight[-1].set_xlim(1, time_final)  # Start from 1 to avoid log(0) issues
  else:
    for ax in axweight:
      ax.set_xscale('linear')
    num_ticks = 6
    x_ticks = np.linspace(0, time_final, num_ticks)
    axweight[-1].set_xticks(x_ticks)
    axweight[-1].set_xticklabels([f'{x:.0f}' for x in x_ticks])
    axweight[-1].set_xlim(-margin, time_final + margin)

  # Adjust layout to remove space between subplots
  plt.subplots_adjust(hspace=0, left=0.15)  # Increased left margin for the new label

  # Add a thin border around the entire figure
  fig.patch.set_linewidth(.5)
  fig.patch.set_edgecolor('black')

  # Add the larger "Weights" label on the left side
  fig.text(0.02, 0.5, 'Weights', va='center', rotation='vertical', fontsize=10)

  # Save and show the figure
  plt.savefig('Primary_Weights.svg', dpi=250, bbox_inches='tight')
  plt.savefig('Primary_Weights.eps', dpi=250, bbox_inches='tight')
  plt.savefig('Primary_Weights.png', dpi=250, bbox_inches='tight')
  plt.show()

# Example usage:
# plot_weights_graphs(100, 1, 1, weight_growth, weight_predation, weight_burst, weight_decay)

# Function to create line graphs that display the total number of relevant processes, as well as the relevant processes per agents color coated in the same manner as the agents in previous graphs.
def processes_graphs(time_list, time_final, processes, tipping_points):
  """
  Plot the number of active terms over time.

  Parameters:
  - time_list (array-like): List of time points.
  - time_final (float): Final time point.
  - processes (array-like): Number of active terms at each time point.
  - processes_phage (array-like): Number of active terms for phage agent at each time point.
  - processes_bacteria (array-like): Number of active terms for bacteria agent at each time point.
  - tipping_points (array-like): List of tipping points.

  Returns:
  None
  """
  # Calculate the figure size
  width_inches = 3.1703
  height_inches = 1.7773

  # Set up the figure and subplots
  fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=300)

  # Set up axes and ticks with DejaVu Sans font
  ax.set_xlabel("Time (h)", fontsize=9, labelpad=2, fontname='DejaVu Sans')  # Changed to 'DejaVu Sans'
  ax.set_ylabel("Number of active terms", fontsize=9, labelpad=2, fontname='DejaVu Sans')  # Changed to 'DejaVu Sans'
  ax.set_xlim(0, time_final)
  ax.set_ylim(0, 7.25)

  # Set y-ticks and labels with DejaVu Sans font
  y_ticks = [0, 1, 2, 3, 4, 5, 6, 7]
  y_tick_labels = [0, 1, 2, 3, 4, "", "", ""]
  ax.set_yticks(y_ticks)
  ax.set_yticklabels(y_tick_labels, fontname='DejaVu Sans')  # Changed to 'DejaVu Sans'

  # Create evenly distributed tick marks for x-axis
  num_ticks = 6
  x_ticks = np.linspace(0, time_final, num_ticks)
  x_ticks[-1] = time_final
  ax.set_xticks(x_ticks)
  ax.set_xticklabels([f'{x:.0f}' for x in x_ticks], fontname='DejaVu Sans')  # Changed to 'DejaVu Sans'

  # Draw vertical lines for tipping points
  for tps in tipping_points:
    index = np.where(time_list == tps)[0]
    if len(index) > 0:
      index = index[0]
      if index > 0:
        ax.vlines(x=tps, ymin=0, ymax=float(7.25),
              color='tab:grey', linestyle='solid', linewidth=1, zorder=1)
      else:
        print(f'Tipping point {tps} is at the start of the time series')
    else:
      print(f'Tipping point {tps} not found in time_list')

  # Plot the main data line
  ax.plot(time_list, processes, label='# of Active Terms', color='k', linewidth=1.15, zorder=2)

  # Adjust tick parameters with DejaVu Sans font
  ax.tick_params(axis='both', which='major', labelsize=7, pad=2)

  plt.tight_layout(pad=0.1)

  # Add a thin border around the entire figure
  fig.patch.set_linewidth(0.5)
  fig.patch.set_edgecolor('black')

  # Save and show the figure
  plt.savefig('processes_graph.svg', dpi=300, bbox_inches='tight')
  plt.savefig('processes_graph.eps', dpi=300, bbox_inches='tight')
  plt.savefig('processes_graph.png', dpi=300, bbox_inches='tight')
  plt.show()

# This function displays the processes between the full and adaptive model; it should be noted that the time steps are based on the observation scale and time_list_adaptive = time_list_full, but there can still be differences in tipping points which is noted here.
def processes_graphs_comparison(time_final, time_list_adaptive, time_list_full, processes_adaptive, processes_full, tipping_points_adaptive, tipping_points_full):
  """
  Display the processes between the full and adaptive model.

  Parameters:
  - time_final (int): The final time of the simulation.
  - time_list_adaptive (array-like): The time steps of the adaptive model.
  - time_list_full (array-like): The time steps of the full model.
  - processes_adaptive (array-like): The number of active terms in the adaptive model.
  - processes_full (array-like): The number of active terms in the full model.
  - tipping_points_adaptive (array-like): The tipping points in the adaptive model.
  - tipping_points_full (array-like): The tipping points in the full model.

  Returns:
  None
  """
  # Calculate the figure size to maintain the 227:114 ratio
  width_inches = 3.504
  height_inches = width_inches * (254 / 313)

  # Set up the figure and subplots
  fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=250)

  ax.plot(time_list_adaptive, processes_adaptive, label='# of Active Terms', color='k', linewidth=0.75)
  ax.plot(time_list_full, processes_full, label='# of Active Terms', color='k', linewidth=0.75)
  ax.plot(time_list_full, )
  ax.set_xlabel("Time (h)", fontsize=7, labelpad=2)
  ax.set_ylabel("Number of active terms", fontsize=7, labelpad=2)
  ax.set_xlim(0, time_final)
  ax.set_ylim(0, 4.25)
  ax.set_yticks([0, 1, 2, 3, 4])

  # Create evenly distributed tick marks for x-axis, ensuring the last tick is at time_final
  num_ticks = 6  # You can adjust this number as needed
  x_ticks = np.linspace(0, time_final, num_ticks)
  x_ticks[-1] = time_final  # Ensure the last tick is exactly at time_final
  ax.set_xticks(x_ticks)
  ax.set_xticklabels([f'{x:.0f}' for x in x_ticks])
  for i, tps in enumerate(tipping_points_adaptive):
    if i == 0:
      if processes_adaptive[np.where(time_list_adaptive == tps)[0][0] - 1] > processes_adaptive[
        np.where(time_list_adaptive == tps)[0][0]]:
        ax.vlines(x=tps, ymin=0, ymax=float(processes_adaptive[np.where(time_list_adaptive == tps)[0][0]]),
              color='tab:grey', linestyle='solid', linewidth=0.75,
              label='Adaptive Model Tipping Point')
      elif processes_adaptive[np.where(time_list_adaptive == tps)[0][0] - 1] < processes_adaptive[
        np.where(time_list_adaptive == tps)[0][0]]:
        ax.vlines(x=tps, ymin=0, ymax=float(processes_adaptive[(np.where(time_list_adaptive == tps)[0][0]) - 1]),
              color='tab:grey', linestyle='solid', linewidth=0.75)
      else:
        print('error')
    else:
      if processes_adaptive[np.where(time_list_adaptive == tps)[0][0] - 1] > processes_adaptive[
        np.where(time_list_adaptive == tps)[0][0]]:
        ax.vlines(x=tps, ymin=0, ymax=float(processes_adaptive[np.where(time_list_adaptive == tps)[0]]),
              color='tab:grey', linestyle='solid', linewidth=0.75)
      elif processes_adaptive[np.where(time_list_adaptive == tps)[0][0] - 1] < processes_adaptive[
        np.where(time_list_adaptive == tps)[0][0]]:
        ax.vlines(x=tps, ymin=0, ymax=float(processes_adaptive[np.where(time_list_adaptive == tps)[0] - 1]),
              color='tab:grey', linestyle='solid', linewidth=0.75)
      else:
        print('error')

  # For full model tipping points
  for i, tps in enumerate(tipping_points_full):
    if i == 0:
      ax.vlines(x=tps, ymin=0, ymax=float(processes_full[np.where(time_list_full == tps)[0][0] - 1]),
            color='k', linestyle=(0, (3, 3, 1, 3)), linewidth=1,
            label='Full Model Tipping Point')
    else:
      ax.vlines(x=tps, ymin=0, ymax=float(processes_full[np.where(time_list_full == tps)[0][0] - 1]),
            color='k', linestyle=(0, (3, 3, 1, 3)), linewidth=1)

  ax.legend(fontsize=6, loc='best')

  # Adjust tick parameters
  ax.tick_params(axis='both', which='major', labelsize=6, pad=2)

  plt.tight_layout(pad=0.1)

  # Add a thin border around the entire figure
  fig.patch.set_linewidth(0.5)
  fig.patch.set_edgecolor('black')

  # Save and show the figure
  plt.savefig('processes_graph.svg', dpi=250, bbox_inches='tight')
  plt.savefig('processes_graph.eps', dpi=250, bbox_inches='tight')
  plt.savefig('processes_graph.png', dpi=250, bbox_inches='tight')
  plt.show()

# Function that categorizes the regime based upon the weights at a given time, utilizing tipping points and trajectories. It also provides a line graph to show the shift in regime over time. One of the outputs of this functions includes the statistical summary of the errors of both the agents and their averages. Finally the values of the error summary for the total scenario are printed in text form.
def regime_finder(Threshold, time_list, final_time, weight_growth, weight_predation, wb, wd, regime_store, tipping_point_list, tipping_points_predation, tipping_points_burst, Bacteria_adaptive, Bacteria_full, Phage_adaptive, Phage_full, p_com_rel_error, p_pha_rel_errors, p_bac_rel_errors):
  """
  Categorizes the regime based on the weights at a given time.

  Parameters:
  - Threshold: The threshold value for categorizing the regime.
  - time_list: List of time points.
  - final_time: The final time point.
  - weight_growth: List of weights for the growth process.
  - weight_predation: List of weights for the predation process.
  - wb: List of weights for the burst process.
  - wd: List of weights for the death process.
  - regime_store: List to store the regime category for each time point.
  - tipping_point_list: List of tipping points.
  - tipping_points_predation: List of tipping points for the predation process.
  - tipping_points_burst: List of tipping points for the burst process.
  - Bacteria_adaptive: List of bacteria population in the adaptive model.
  - Bacteria_full: List of bacteria population in the full model.
  - Phage_adaptive: List of phage population in the adaptive model.
  - Phage_full: List of phage population in the full model.
  - p_com_rel_error: List of relative errors for the combined population.
  - p_pha_rel_errors: List of relative errors for the phage population.
  - p_bac_rel_errors: List of relative errors for the bacteria population.

  Returns:
  - regime_store: List of regime categories for each time point.
  - regime_indices: Dictionary of regime indices for each regime category.
  - regime_stats: Dictionary of statistical summaries for each regime category.
  """
  regime_indices = {i: [] for i in range(1, 17)}  # Dictionary to store indices for each regime

  for times in range(len(time_list)):
    if weight_growth[times] >= Threshold and weight_predation[times] < Threshold and wb[times] < Threshold and wd[times] < Threshold:
      regime_store_i = 1
    elif weight_growth[times] >= Threshold and weight_predation[times] < Threshold and wb[times] >= Threshold and wd[times] < Threshold:
      regime_store_i = 2
    elif weight_growth[times] >= Threshold and weight_predation[times] >= Threshold and wb[times] >= Threshold and wd[times] < Threshold:
      regime_store_i = 3
    elif weight_growth[times] >= Threshold and weight_predation[times] >= Threshold and wb[times] < Threshold and wd[times] < Threshold:
      regime_store_i = 4
    elif weight_growth[times] < Threshold and weight_predation[times] < Threshold and wb[times] >= Threshold and wd[times] >= Threshold:
      regime_store_i = 5
    elif weight_growth[times] < Threshold and weight_predation[times] >= Threshold and wb[times] >= Threshold and wd[times] >= Threshold:
      regime_store_i = 6
    elif weight_growth[times] < Threshold and weight_predation[times] >= Threshold and wb[times] < Threshold and wd[times] >= Threshold:
      regime_store_i = 7
    elif weight_growth[times] < Threshold and weight_predation[times] < Threshold and wb[times] < Threshold and wd[times] >= Threshold:
      regime_store_i = 8
    elif weight_growth[times] >= Threshold and weight_predation[times] < Threshold and wb[times] >= Threshold and wd[times] >= Threshold:
      regime_store_i = 9
    elif weight_growth[times] >= Threshold and weight_predation[times] >= Threshold and wb[times] >= Threshold and wd[times] >= Threshold:
      regime_store_i = 10
    elif weight_growth[times] >= Threshold and weight_predation[times] >= Threshold and wb[times] < Threshold and wd[times] >= Threshold:
      regime_store_i = 11
    elif weight_growth[times] >= Threshold and weight_predation[times] < Threshold and wb[times] < Threshold and wd[times] >= Threshold:
      regime_store_i = 12
    elif weight_growth[times] < Threshold and weight_predation[times] < Threshold and wb[times] >= Threshold and wd[times] < Threshold:
      regime_store_i = 13
    elif weight_growth[times] < Threshold and weight_predation[times] >= Threshold and wb[times] >= Threshold and wd[times] < Threshold:
      regime_store_i = 14
    elif weight_growth[times] < Threshold and weight_predation[times] >= Threshold and wb[times] < Threshold and wd[times] < Threshold:
      regime_store_i = 15
    elif weight_growth[times] < Threshold and weight_predation[times] < Threshold and wb[times] < Threshold and wd[times] < Threshold:
      regime_store_i = 16

    regime_store.append(regime_store_i)
    regime_indices[regime_store_i].append(times)

  # Calculate statistics for each regime
  regime_stats = {}
  for regime, indices in regime_indices.items():
    if indices:
      # Use numpy array indexing to access elements using a list of indices
      com_data = np.array(p_com_rel_error)[indices]
      pha_data = np.array(p_pha_rel_errors)[indices]
      bac_data = np.array(p_bac_rel_errors)[indices]

      regime_stats[regime] = {
        'com': {
          'min': np.min(com_data),
          'q1': np.percentile(com_data, 25),
          'median': np.median(com_data),
          'mean': np.mean(com_data),
          'q3': np.percentile(com_data, 75),
          'max': np.max(com_data)
        },
        'pha': {
          'min': np.min(pha_data),
          'q1': np.percentile(pha_data, 25),
          'median': np.median(pha_data),
          'mean': np.mean(pha_data),
          'q3': np.percentile(pha_data, 75),
          'max': np.max(pha_data)
        },
        'bac': {
          'min': np.min(bac_data),
          'q1': np.percentile(bac_data, 25),
          'median': np.median(bac_data),
          'mean': np.mean(bac_data),
          'q3': np.percentile(bac_data, 75),
          'max': np.max(bac_data)
        }
      }

  # Plotting code (unchanged)
  plt.figure()
  fig, (axpop, axregimes) = plt.subplots(2, 1, sharex=True)
  axpop.set_title('Population Model Versus Time', fontsize=7)
  axpop.set_ylabel('Populations Concentration units/mL', fontsize=6)
  axpop.set_xlabel('Time (h)', fontsize=6)
  axpop.set_xlim(0, final_time)
  if round(np.log10(max(Phage_full))) > round(np.log10(max(Bacteria_full))):
    max_limit = (1000 * (10**round(np.log10(max(Phage_full)))))
    axpop.set_ylim(1, (1000 * (10**round(np.log10(max(Phage_full))))))
  else:
    max_limit = (1000 * (10**round(np.log10(max(Bacteria_full)))))
    axpop.set_ylim(1, (1000 * (10**round(np.log10(max(Bacteria_full))))))
  axpop.semilogy(time_list, Bacteria_full, color='b', linestyle='solid')
  axpop.semilogy(time_list, Phage_full, color='r', linestyle='solid')
  axpop.semilogy(time_list, Bacteria_adaptive, color='k', linestyle='dashed')
  axpop.semilogy(time_list, Phage_adaptive, color='k', linestyle='dashed')
  for tps in tipping_point_list:
    if tps in tipping_points_predation:
      axpop.vlines(x=tps, ymin=0, ymax=(float(Phage_adaptive[([np.where(time_list == tps)])[0][0][0]])), color='k', linestyle='dotted')
    elif tps in tipping_points_burst:
      axpop.vlines(x=tps, ymin=0, ymax=(float(Bacteria_adaptive[([np.where(time_list == tps)])[0][0][0]])), color='k', linestyle='dotted')
    else:
      print('error')
  axregimes.set_title('Regime Category Versus Time', fontsize=7)
  axregimes.set_xlabel('Time (h)', fontsize=6)
  axregimes.set_ylabel('Regime Category', fontsize=6)
  axregimes.set_yticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI'], fontsize=5)
  line_spots = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
  for spots in line_spots:
    axregimes.axhline(y=spots, color='tab:gray', linestyle='dotted')
  axregimes.set_ylim(0, 16)
  axregimes.scatter(time_list, regime_store, c='blue', s=0.5)
  for tps in tipping_point_list:
    axregimes.axvline(x=tps, color='k')
  plt.tight_layout()
  plt.savefig('Regime Category Versus Time')

  regime_indices[0] = list(range(len(time_list)))  # Add index for all regimes
  regime_stats[0] = {
    'com': {
      'min': np.min(p_com_rel_error),
      'q1': np.percentile(p_com_rel_error, 25),
      'median': np.median(p_com_rel_error),
      'mean': np.mean(p_com_rel_error),
      'q3': np.percentile(p_com_rel_error, 75),
      'max': np.max(p_com_rel_error)
    },
    'pha': {
      'min': np.min(p_pha_rel_errors),
      'q1': np.percentile(p_pha_rel_errors, 25),
      'median': np.median(p_pha_rel_errors),
      'mean': np.mean(p_pha_rel_errors),
      'q3': np.percentile(p_pha_rel_errors, 75),
      'max': np.max(p_pha_rel_errors)
    },
    'bac': {
      'min': np.min(p_bac_rel_errors),
      'q1': np.percentile(p_bac_rel_errors, 25),
      'median': np.median(p_bac_rel_errors),
      'mean': np.mean(p_bac_rel_errors),
      'q3': np.percentile(p_bac_rel_errors, 75),
      'max': np.max(p_bac_rel_errors)
    }
  }
  regime_indices['Culmination'] = regime_indices[0]  # Add index for the culmination of the scenario
  regime_stats['Culmination'] = regime_stats[0]  # Add statistical summary for the culmination of the scenario

  return regime_store, regime_indices, regime_stats

# Function that displays the relative error as a line graph, displaying both the tipping points of both models (adaptive and full model).
def relative_error_graphs(time_list, time_final, p_bac_relative_error_percent, p_pha_relative_error_percent,p_com_relative_error_percent, tipping_point_list_adaptive, tipping_point_list_full):
  """
  Function to plot the relative error as a line graph and display the tipping points of both the adaptive and full models.

  Parameters:
  - time_list (list): List of time values.
  - time_final (int): Final time value.
  - p_bac_relative_error_percent (list): List of relative error percentages for bacteria.
  - p_pha_relative_error_percent (list): List of relative error percentages for phage.
  - p_com_relative_error_percent (list): List of overall relative error percentages.
  - tipping_point_list_adaptive (list): List of tipping points for the adaptive model.
  - tipping_point_list_full (list): List of tipping points for the full model.

  Returns:
  - p_com_relative_error_percent (list): List of overall relative error percentages.
  """

  for t in range(len(p_bac_relative_error_percent)):
      p_com_relative_error_percent_ind = np.mean([p_bac_relative_error_percent[t],p_pha_relative_error_percent[t]])
      p_com_relative_error_percent.append(p_com_relative_error_percent_ind)

  # Calculate the figure size to maintain the 227:127 ratio
  width_inches = 3.632
  height_inches = width_inches * (114 / 215)

  # Set up the figure and subplots
  fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=250)

  ax.plot(time_list, p_bac_relative_error_percent, label='Relative Error Bacteria', color='b', linewidth=0.75)
  ax.plot(time_list, p_pha_relative_error_percent, label='Relative Error Phage', color='r', linewidth=0.75)
  ax.plot(time_list, p_com_relative_error_percent, label = "Relative Error", color = '#4f5050', linewidth=0.8)

  ax.set_xlabel("Time (h)", fontsize=7, labelpad=2)
  ax.set_ylabel("Relative Error (%)", fontsize=7, labelpad=2)
  ax.set_xlim(0, time_final)

  # Create evenly distributed tick marks for x-axis
  num_ticks = 6  # You can adjust this number as needed
  x_ticks = np.linspace(0, time_final, num_ticks)
  ax.set_xticks(x_ticks)
  ax.set_xticklabels([f'{x:.0f}' for x in x_ticks])

  for tps in tipping_point_list_adaptive:
      if tps == tipping_point_list_adaptive[0]:
          ax.axvline(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75, label='Adaptive Model Tipping Point')
      else:
          ax.axvline(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)
  for tps in tipping_point_list_full:
      if tps == tipping_point_list_full[0]:
          ax.axvline(x=tps, ymin=0, ymax=1, color='k', linestyle=(0, (3, 3, 0, 0)), linewidth=1, label='Full Model Tipping Point')
      else:
          ax.axvline(x=tps, ymin=0, ymax=1, color='k', linestyle=(0, (3, 3, 0, 0)), linewidth=1)

  ax.legend(fontsize=6, loc='best')

  # Adjust tick parameters
  ax.tick_params(axis='both', which='major', labelsize=6, pad=2)

  plt.tight_layout(pad=0.1)

  # Add a thin border around the entire figure
  fig.patch.set_linewidth(0.5)
  fig.patch.set_edgecolor('black')

    # Calculate the maximum relative error
  max_relative_error = max(max(p_bac_relative_error_percent), max(p_pha_relative_error_percent))

  # Set up the main plot
  ax.set_ylim(0, 15)
  ax.set_yticks([0, 5, 10, 15])
  ax.set_yticklabels(['0', '5', '10', '15'])

    # Create an inlay if the maximum relative error exceeds 15%
  if max_relative_error > 15:
      # Find the last tipping point
      last_tipping_point = max(max(tipping_point_list_adaptive or [0]), max(tipping_point_list_full or [0]))

      # Calculate inlay position (between last tipping point and end of graph)
      inlay_left = 0.5 + (0.45 * last_tipping_point / time_final)
      inlay_width = 0.45 * (1 - last_tipping_point / time_final)

      # Create an inlay axes
      inlay_ax = fig.add_axes([inlay_left, 0.65, inlay_width, 0.25])  # [left, bottom, width, height]

      # Plot the full range data in the inlay
      inlay_ax.plot(time_list, p_bac_relative_error_percent, color='b', linewidth=0.5)
      inlay_ax.plot(time_list, p_pha_relative_error_percent, color='r', linewidth=0.5)
      inlay_ax.plot(time_list, p_com_relative_error_percent, color='#4f5050', linewidth=0.5)

      # Set the y-axis limits for the inlay
      inlay_ax.set_ylim(0, max(50, max_relative_error * 1.1))  # Set upper limit to at least 100 or 10% above max error

      # Set x-axis limits for the inlay
      inlay_ax.set_xlim(0, time_final)

      # Set x-ticks for the inlay (similar to main plot)
      inlay_ax.set_xticks(x_ticks)
      inlay_ax.set_xticklabels([f'{x:.0f}' for x in x_ticks], fontsize=5)

      #set y-ticks for inlay
      inlay_ax.set_yticks([0, 25, 50])
      inlay_ax.set_yticklabels(['0', '25', '50'])

      # Adjust tick parameters for the inlay
      inlay_ax.tick_params(axis='both', which='major', labelsize=5)

  # Save the figure with inlay (if applicable)
  plt.savefig('relative_error_with_inlay.svg', dpi=250, bbox_inches='tight')
  plt.show()
  return [p_com_relative_error_percent]



  #differential functions including the adaptive and full equations (standard lokta-volterra)
# This is the function used for the adaptive model: it utilizes thetas to reflect the point at which the weights of each process go over the threshold level. This is an input to the solve_ivp function which integrates based on a chosen method (LSODA is the one used within the main text and supplemental text).

def theta_approach_model(t, populations, rt, at, ct, mt, tobst, reflevelt, activethrest):
  """
  Calculate the population dynamics using the adaptive model.

  Parameters:
    t (float): Time.
    populations (list): List of population values [btest, ptest].
    rt (float): Growth rate of bacteria.
    at (float): Attachment rate of phage to bacteria.
    ct (float): Conversion rate of attached phage to bacteria.
    mt (float): Mortality rate of phage.
    tobst (float): Observation time scale.
    reflevelt (float): Reference level for threshold calculation.
    activethrest (float): Threshold level for activation.

  Returns:
    list: List of population derivatives [db, dp].
  """

  btest, ptest = populations
  theta_g_1 = theta_p_1 = theta_b_1 = theta_d_1 = 1

  if ((tobst * rt * btest) / (reflevelt * btest)) < activethrest:
    theta_g_1 = 0

  if ((tobst * at * ptest * btest) / (reflevelt * btest)) < activethrest:
    theta_p_1 = 0

  if ((tobst * ct * at * ptest * btest) / (reflevelt * ptest)) < activethrest:
    theta_b_1 = 0

  if ((tobst * mt * ptest) / (ptest * reflevelt)) < activethrest:
    theta_d_1 = 0

  db = ((theta_g_1 * rt * btest) - (theta_p_1 * at * btest * ptest))
  dp = ((theta_b_1 * ct * at * btest * ptest) - (theta_d_1 * mt * ptest))

  return [db, dp]
# This is the function used for the full (standard) model, to ensure it acts as a control, theta is still used but it is equivalent to 1 at all times thus making all processes relevant.
def lotka_volterra_standard(t, populations_standard, rt, at, ct, mt, tobst, reflevelt, activethrest):
  """
  Calculate the population dynamics using the full (standard) model.

  Parameters:
    t (float): Time.
    populations_standard (list): List of population values [bstandard, pstandard].
    rt (float): Growth rate of bacteria.
    at (float): Attachment rate of phage to bacteria.
    ct (float): Conversion rate of attached phage to bacteria.
    mt (float): Mortality rate of phage.
    tobst (float): Observation time scale.
    reflevelt (float): Reference level for threshold calculation.
    activethrest (float): Threshold level for activation.

  Returns:
    list: List of population derivatives [dbstandard, dpstandard].
  """
  bstandard, pstandard = populations_standard
  theta_g_1 = theta_p_1 = theta_b_1 = theta_d_1 = 1
  dbstandard = ((theta_g_1 * rt * bstandard) - (theta_p_1 * at * bstandard * pstandard))
  dpstandard = ((theta_b_1 * ct * at * bstandard * pstandard) - (theta_d_1 * mt * pstandard))
  return [dbstandard, dpstandard]
# Artifiact of a previous function
def handle_tipping_points(tipping_points):
  if len(tipping_points) == 0:
    return None
  elif len(tipping_points) == 1:
    return tipping_points[0]
  else:
    return tipping_points

# Function used to generate a phase diagram of the dynamics within a scenario, in which the critical values generate horizontal and vertical lines that implicitly demonstrates the regimes of scenario as crossing them demonstrates when processes become relevant or irrelevant.
def phase_diagram_maker(Agent_Phage, Agent_Phage_Simplified, Agent_Bacteria, Agent_Bacteria_Simplified, para_test):
    plt.loglog(Agent_Bacteria, Agent_Phage, color='k')
    plt.loglog(Agent_Bacteria_Simplified, Agent_Phage_Simplified, color='tab:grey', linestyle=(0, (5, 10)))
    pointsx = [Agent_Bacteria[0], Agent_Bacteria[-1]]
    pointsy = [Agent_Phage[0], Agent_Phage[-1]]
    plt.xlabel('Prey Concentration ($ml^{-1}$)', fontsize=10)
    plt.ylabel('Predator Concentration ($ml^{-1}$)', fontsize=10)
    plt.axhline(y=[(para_test[5]*1)/((para_test[1])*(para_test[4]))], xmin=0.03, xmax=0.98, color="red")
    plt.axvline(x=[(para_test[5]*1)/((para_test[2])*(para_test[4])*(para_test[1]))], ymin=0.03, ymax=0.98, color="blue")
    plt.scatter(pointsx, pointsy, c='tab:grey', s=100)
    plt.minorticks_off()
    # Add labels to the first and last points
    plt.annotate('t0', (pointsx[0], pointsy[0]), xytext=(6, 6), textcoords='offset points')
    plt.annotate('tf', (pointsx[1], pointsy[1]), xytext=(6, 6), textcoords='offset points')

    plt.tight_layout()  # Adjust layout to fit everything
    plt.savefig('phase.svg', format='svg', dpi=300, bbox_inches='tight')
    plt.savefig('phase.eps', format='eps', dpi=300, bbox_inches='tight')
    plt.savefig('phase.pdf', format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Similar to the previous function, but it is able to show an additional set of initial values within the same phase diagram, the second set of values is depicted in green.
def phase_diagram_maker_with_contained(Agent_Phage, Agent_Phage_Simplified, Agent_Bacteria, Agent_Bacteria_Simplified, para_test, contained_phage, contained_phage_simplified, contained_bacteria, contained_bacteria_simplified):
    plt.loglog(Agent_Bacteria, Agent_Phage, color='k')
    plt.loglog(Agent_Bacteria_Simplified, Agent_Phage_Simplified, color='tab:grey', linestyle=(0, (5, 10)))
    plt.loglog(contained_bacteria, contained_phage, color='tab:green')
    plt.loglog(contained_bacteria_simplified, contained_phage_simplified, color='tab:green', linestyle=(0, (3, 15)))
    pointsx = [Agent_Bacteria[0], Agent_Bacteria[-1], contained_bacteria[0], contained_bacteria[-1]]
    pointsy = [Agent_Phage[0], Agent_Phage[-1], contained_phage[0], contained_phage[-1]]
    plt.xlabel('Prey Concentration ($ml^{-1}$)', fontsize=10)
    plt.ylabel('Predator Concentration ($ml^{-1}$)', fontsize=10)
    plt.axhline(y=[(para_test[5]*1)/((para_test[1])*(para_test[4]))], xmin=0.03, xmax=0.98, color="red")
    plt.axvline(x=[(para_test[5]*1)/((para_test[2])*(para_test[4])*(para_test[1]))], ymin=0.03, ymax=0.98, color="blue")
    plt.scatter(pointsx[0:2], pointsy[0:2], c='tab:grey', s=100)
    plt.scatter(pointsx[2:4], pointsy[2:4], c='tab:green', s=100)
    plt.minorticks_off()

    # Add labels to the first and last points
    plt.annotate('t0', (pointsx[0], pointsy[0]), xytext=(6, 6), textcoords='offset points')
    plt.annotate('tf', (pointsx[1], pointsy[1]), xytext=(6, 6), textcoords='offset points')
    plt.annotate('t0', (pointsx[2], pointsy[2]), xytext=(6, 6), textcoords='offset points')
    plt.annotate('tf', (pointsx[3], pointsy[3]), xytext=(6, 6), textcoords='offset points')

    plt.tight_layout()  # Adjust layout to fit everything
    plt.savefig('phase_contained.svg', format='svg', dpi=300, bbox_inches='tight')
    plt.savefig('phase_contained.eps', format='eps', dpi=300, bbox_inches='tight')
    plt.savefig('phase_contained.pdf', format='pdf', dpi=300, bbox_inches='tight')
    plt.show()
# Function used to calculate the difference between the tipping points of the adaptive model and the full model.
def tipping_point_error_finder(tp_full,tp):
  for i in range(0, len(tp_full)):
    print(f'Time error between the regime ending at {tp_full[i]}:{100 * (abs(tp_full[i]-tp[i])/tp_full[i])}')

## Storage lists utilized

In [ ]:
#List weights of all processes for the adaptive model
wg = wp = wb = wd = []
#List weights of all processes for the full model
wg_full = wp_full = wb_full = wd_full = []
#List of tipping points for the adaptive model
tp_g = tp_p = tp_b = tp_d = []
#List of tipping points for the full model
tp_g_full = tp_p_full = tp_b_full = tp_d_full = []
#List of all processes for the adaptive model per time step
pro = []
#List of all processes for the full model per time step
pro_full = []
#List for all of the error types calculated in bacterial population per time step between the two models
p_error = p_rel_error = p_rel_error_percent = []
#List for all of the error types calculated in phage population per time step between the two models
b_error = b_rel_error = b_rel_error_percent = []
#List to place the values of all bacterial agent processes per time step
processes_bacteria = []
#List to place the values of all phage agent processes per time step
processes_phage = []
#List for the relative error of the combined population per time step
p_com_relative_error_percent = []
#List for the relative error of the phage population per time step
regime_store =[]


# Model Simulation: Standard Lotka-Volterra
## ODE solvers and solutions to model

In [ ]:
# Initial conditions for the system, set by initial selected parameters
y0 = [(bi),(pi)]

# Generating the time steps used by both of the solvers, required to use the solve_ivp() function.
timespan = (0,tobs)
t_for_solver = np.linspace(0, tobs,10000)

# ODE solvers, both the models are used with the same initial conditions and parameters. Warning do not set the tolerance any lower than 10^-13 as it will cause the solver to fail.
## Solving the adaptive model using the solve_ivp() function
sol_adapt_new_module = solve_ivp(theta_approach_model,timespan,y0=y0,args=para_test,method= 'LSODA',t_eval=t_for_solver,rtol = 10**-13,atol= 10**-16)

## Solving the full model using the solve_ivp() function
sol_full_new_module = solve_ivp(lotka_volterra_standard,timespan,y0=y0,args=para_test,method= 'LSODA',t_eval=t_for_solver,rtol = 10**-13,atol= 10**-16)

## Outputting the times of both models, these times should be the same but the output was used to confirm
t = sol_adapt_new_module.t
t_full = sol_full_new_module.t

## The dynamics of both agents generated by the adaptive model
b = sol_adapt_new_module.y[0,:]
p = sol_adapt_new_module.y[1,:]

## The dynamics of both agents generated by the full model
b_full = sol_full_new_module.y[0,:]
p_full = sol_full_new_module.y[1,:]

## Calculation of weights per processes, tipping points and errors for adaptive and full model
Running this code generates weights and tipping points for both models, and errors analysis (absolute error, and relative error) between the two models.

In [ ]:
##Weight_functions for adaptive##
wg_function = para_test[4] * (para_test[0] * b/(para_test[5] * b))
wp_function = para_test[4] * ((para_test[1] * p * b)/(para_test[5] * b))
wb_function = para_test[4] * ((para_test[2] * para_test[1] * p * b)/(para_test[5] * p))
wd_function = para_test[4] * ((para_test[3] * p)/(para_test[5] * p))
##Weight_functions for full##
wg_function_full = para_test[4] * (para_test[0] * b_full/(para_test[5] * b_full))
wp_function_full = para_test[4] * ((para_test[1] * p_full * b_full)/(para_test[5] * b_full))
wb_function_full = para_test[4] * ((para_test[2] * para_test[1] * p_full * b_full)/(para_test[5] * p_full))
wd_function_full = para_test[4] * ((para_test[3] * p_full)/(para_test[5] * p_full))
#Finding the weights for each adaptive process
wg = weights_finder(wg_function,t,wg)
wp = weights_finder(wp_function,t,wp)
wb = weights_finder(wb_function,t,wb)
wd = weights_finder(wd_function,t,wd)
#Finding the weights for each adaptive process
wg_full = weights_finder(wg_function_full,t,wg_full)
wp_full = weights_finder(wp_function_full,t,wp_full)
wb_full = weights_finder(wb_function_full,t,wb_full)
wd_full = weights_finder(wd_function_full,t,wd_full)
#Finding the tipping points for each adaptive process, if there is one
tp_g = tipping_point_finder(wg,t,tp_g,para_test[6])
tp_p = tipping_point_finder(wp,t,tp_p,para_test[6])
tp_b = tipping_point_finder(wb,t,tp_b,para_test[6])
tp_d = tipping_point_finder(wd,t,tp_d,para_test[6])
tp = tp_g + tp_p + tp_b + tp_d
#Finding the tipping points for each full process, if there is one
tp_g_full = tipping_point_finder(wg_full,t_full,tp_g_full,para_test[6])
tp_p_full = tipping_point_finder(wp_full,t_full,tp_p_full,para_test[6])
tp_b_full = tipping_point_finder(wb_full,t_full,tp_b_full,para_test[6])
tp_d_full = tipping_point_finder(wd_full,t_full,tp_b_full,para_test[6])
tp_full = tp_g_full + tp_p_full + tp_b_full + tp_d_full
#Finding the adaptive processes that are active per each individual time unit
pro = process_counter(para_test[6],wg,wp,wb,wd,t,pro)
#Finding the full processes that are active per each individual time unit
pro_full = process_counter(para_test[6],wg_full,wp_full,wb_full,wd_full,t_full,pro_full)
#finding the error,relative error, relative error percent, and lopez's error
p_error = true_error(t,p_full,p,p_error)
p_rel_error = relative_error(t,p_error,p_full,p_rel_error)
p_rel_error_percent = relative_error_percentage_finder(p_rel_error,t,p_rel_error_percent)
b_error = true_error(t,b_full,b,b_error)
b_rel_error = relative_error(t,b_error,b_full,b_rel_error)
b_rel_error_percent = relative_error_percentage_finder(b_rel_error,t,b_rel_error_percent)

# Visualization of the models and model analysis
## Population vs time graphs of full and adaptive model

In [ ]:
population_plot_comparison_and_tipping_points(t_full,tobs,tp_full,tp_p_full,tp_b_full,p_full,b_full,p,b)

## Critical values calculations for predation and burst process
Running this code generates the exact value that triggered a tipping point in the opposite agent population.

In [ ]:
# Displays the critical values that yield the regimes as seen in population plots above
print(f'Predation Critical Value: {(1 * 0.10) /(tobs * a)} phages/ml')
print(f'Burst Critical Value: {(1 * 0.10) /(tobs * c * a)} cells/ml')

## Weight subplots for processes within full model
Running the code below will generate a figure displaying each process as a subplot in a single figure.

In [ ]:
plot_weights_graphs(tobs,para_test[6],para_test[5],wg_full,wp_full,wb_full,wd_full, tp_p_full, tp_b_full)

## Relevant processes vs time graph for full model
Running this code generates a process vs time graph from the full model

In [ ]:
processes_graphs(t_full,tobs,pro_full,tp_full)

## Relative error vs time graph for both agents
Running this code generates a relative error vs time graph for both population as well as an average error of the scenario.

In [ ]:
relative_error_graphs(t,tobs,b_rel_error_percent,p_rel_error_percent,p_com_relative_error_percent,tp, tp_full)

## Tipping point relative error percent between full and adaptive model

Running this code generates and displays the relative error percent between full and adaptive model

In [ ]:
tipping_point_error_finder(tp_full,tp)

## Phase diagram of the scenario
Running this code will generate the phase diagram of the scenario (the population of both agents, and predicted critical values for predation and burst)

In [ ]:
phase_diagram_maker(p_full, p, b_full,b, para_test)

## Regime versus time graph and error analysis per regime
Running this code generates an population versus time graph with a corresponding regime graph underneath. In addition to indexes displaying the error anaylsis: minimum, first quartile, median, mean, third quartile, and maximume relative error percent per regime are displayed.

In [ ]:
regime_finder(thresholdlevel,t,tobs,wg,wp,wb,wd,regime_store,tp,tp_p,tp_b,b,b_full,p,p_full,p_com_relative_error_percent,  p_rel_error_percent, b_rel_error_percent)

# Storaging outputs
## Storage of arrays related to time-steps
Running this code places all arrays and lists into a singular .xslx sheet titled data_output.xslx

In [ ]:
# Create a new DataFrame with the required data
df_for_xslx = pd.DataFrame({
    'time': t_full,
    'time_adaptive': t,
    'phage_full': p_full,
    'bacteria_full': b_full,
    'phage_adaptive': p,
    'bacteria_adaptive': b,
    'process_full': pro_full,
    'processes': pro,
    'weight_growth_full': wg_full,
    'weight_predation_full': wp_full,
    'weight_burst_full': wb_full,
    'weight_decay_full': wd_full,
    'weight_growth': wg,
    'weight_predation': wp,
    'weight_burst': wb,
    'weight_decay': wd,
    'error_phage': p_error,
    'error_bacteria': b_error,
    'rel_error_phage': p_rel_error,
    'rel_error_bacteria': b_rel_error,
    'rel_error_phage_per': p_rel_error_percent,
    'rel_error_bac_per': b_rel_error_percent,
    'rel_error_com_per': p_com_relative_error_percent,
    'regime': regime_store
})

# Define the name of the Excel file to be created
excel_file = 'data_output.xlsx'

# Export the DataFrame to an Excel file
df_for_xslx.to_excel(excel_file, index=False, sheet_name='Sheet1')

## Storage of initial conditions and predicted tipping points
Running this code saves the initial parameters inputted to simulate scenario and predicted tipping points into a .txt file titled output.txt

In [ ]:
df_to_txt = pd.DataFrame({'Time Observed': [tobs], 'Bacterial Growth Rate':[r],'Phage Adsorption Rate':[a],'Burst Size':[c],'Phage Decay Rate':[m],'Threshold': [thresholdlevel], 'Reference Level': [reference_level], 'Tipping Points': [tp], 'Tipping Points for Burst Process': [tp_b], 'Tipping Points for Predation Process': [tp_p]})

# Convert DataFrame to string
df_string = df_to_txt.to_string(index=False)

# Write to a text file
with open('output.txt', 'w') as f:
    f.write(df_string)

# Scenario comparison simulation and analysis
## Input parameters
Running this code generates a second set of full and adaptive models based upon set inputs. Within the paper (Cobo-Lopez et al., 2025), this input was either the Scenario 3 quasi-equilibrium values or Scenario 4 quasi-static values.

In [ ]:
# parameters
# growth parameter of bacteria
r_contained = 0.9  # The growth rate of bacteria in the scenario

a_contained = (3.0 * 10**-8)  # The rate at which phages can enter a bacteria in the scenario

c_contained = 50  # The burst size of phages in the scenario

m_contained = (1.5 * 10**-3)  # The decay rate of phages in the scenario

tobs_contained = 14  # The observed time period in the scenario

bi_contained = 10**3  # The initial population of bacteria in the scenario

pi_contained = 10**4  # The initial population of phages in the scenario

reference_level_contained = 0.10  # The reference level for tipping points in the scenario

thresholdlevel_contained = 1.0  # The threshold level for tipping points in the scenario

## ODE solver and solution for secondary scenario input
Running this code solves a second set of ODE Lotka-Volterra models and stores the solution.

In [ ]:
para_test_contained = (r_contained, a_contained, c_contained, m_contained, tobs_contained, reference_level_contained, thresholdlevel_contained)  # Parameters for the secondary scenario

y0_contained = [(bi_contained), (pi_contained)]  # Initial conditions for the system

timespan_contained = (0, tobs_contained)  # Time span for the simulation

t_for_solver_contained = np.linspace(0, tobs_contained, 10000)  # Time steps used by the solvers

sol_adapt_new_module_contained = solve_ivp(theta_approach_model, timespan_contained, y0=y0_contained, args=para_test_contained, method='LSODA', t_eval=t_for_solver_contained, rtol=10**-13, atol=10**-16)  # Solution of the adaptive model

sol_full_new_module_contained = solve_ivp(lotka_volterra_standard, timespan_contained, y0=y0_contained, args=para_test_contained, method='LSODA', t_eval=t_for_solver_contained, rtol=10**-13, atol=10**-16)  # Solution of the full model

t_contained = sol_adapt_new_module.t  # Time array for the adaptive model

t_full_contained = sol_full_new_module.t  # Time array for the full model

b_contained = sol_adapt_new_module_contained.y[0, :]  # Population of bacteria for the adaptive model

p_contained = sol_adapt_new_module_contained.y[1, :]  # Population of phages for the adaptive model

b_full_contained = sol_full_new_module_contained.y[0, :]  # Population of bacteria for the full model

p_full_contained = sol_full_new_module_contained.y[1, :]  # Population of phages for the full model

## Visualizing both scenarios as a single phase diagram
Running this code builds a phase diagram with the intial scenario in black and grey, and overlays the secondary scenario in green for ease in comparison.

In [ ]:
phase_diagram_maker_with_contained(p_full, p, b_full,b, para_test, p_full_contained, p_contained,b_full_contained, b_contained)

# Modified model: Lotka-Volterra model with carrying capacity
## Modified model parameter inputs

In [ ]:
# Modified model parameter inputs for carrying capacity model
r_cc = 0.1  # growth parameter of bacteria
a_cc = (3.0*10**-9)  # the rate at which phages can enter a bacteria
c_cc = 50  # the burst size of phages
m_cc = (0.1)  # the decay rate of phages
tobs_cc = 30000  # the observed time period
k_cc = 10**9  # the carrying capacity of the environment
bi_cc = 6*10**5  # the initial population of bacteria
pi_cc = 6*10**6  # the initial population of phages
reference_level_cc = 0.10  # the reference level for tipping points
thresholdlevel_cc = 1.0  # the threshold level for tipping points
para_test_cc = (r_cc,a_cc,c_cc,m_cc,k_cc,tobs_cc,reference_level_cc,thresholdlevel_cc) # parameters for the carrying capacity model

## Functions utilized to analyze and visualize Lotka-Volterra model with carrying capacity

In [ ]:
# Function for the input of a carrying capacity within the theta model
def theta_approach_model_carrying_capacity(t, populations_cc, rt, at, ct, mt, kt, obst, reflevelt, activethrest):
    """
    Function for the input of a carrying capacity within the theta approach model.

    Parameters:
    - t: (array-like) time values
    - populations_cc: List of current populations [bacteria, phage]
    - rt: Growth rate of bacteria
    - at: Predation rate of phage on bacteria
    - ct: Burst rate of phage
    - mt: Decay rate of phage
    - kt: Carrying capacity of bacteria
    - obst: Observed time
    - reflevelt: Reference level of bacteria
    - activethrest: Threshold for activation

    Returns:
    - List of derivatives [db_c, dp_c] representing the rate of change of bacteria and phage populations.
    """
    b_cc, p_cc = populations_cc
    theta_gcc_1 = theta_pcc_1 = theta_bcc_1 = theta_dcc_1 = theta_cc_1 = 1

    # Check if the growth rate of bacteria is below the activation threshold
    if ((obst * rt * b_cc) / (reflevelt * b_cc)) < activethrest:
        theta_gcc_1 = 0

    # Check if the predation rate of phage on bacteria is below the activation threshold
    if ((obst * at * p_cc * b_cc) / (reflevelt * b_cc)) < activethrest:
        theta_pcc_1 = 0

    # Check if the burst rate of phage is below the activation threshold
    if ((obst * ct * at * p_cc * b_cc) / (reflevelt * p_cc)) < activethrest:
        theta_bcc_1 = 0

    # Check if the decay rate of phage is below the activation threshold
    if ((obst * mt * p_cc) / (p_cc * reflevelt)) < activethrest:
        theta_dcc_1 = 0

    # Check if the carrying capacity of bacteria is below the activation threshold
    if ((obst * rt * b_cc**2) / (reflevelt * b_cc * kt)) < activethrest:
        theta_cc_1 = 0

    # Calculate the derivatives of bacteria and phage populations
    db_c = ((theta_gcc_1 * rt * b_cc) - (theta_pcc_1 * at * b_cc * p_cc) - ((theta_cc_1 * rt * b_cc**2) / (kt)))
    dp_c = ((theta_bcc_1 * ct * at * b_cc * p_cc) - (theta_dcc_1 * mt * p_cc))

    return [db_c, dp_c]

# Function for the input of a carrying capacity within the full model
def lotka_volterra_standard_cc(t, populations_standard_cc, rt, at, ct, mt, kt, tobst, reflevelt, activethrest):
    """
    Function for the input of a carrying capacity within the standard Lotka-Volterra model.

    Parameters:
    - t: (array-like) time values
    - populations_standard_cc: List of current populations [bacteria, phage]
    - rt: Growth rate of bacteria
    - at: Predation rate of phage on bacteria
    - ct: Burst rate of phage
    - mt: Decay rate of phage
    - kt: Carrying capacity of bacteria
    - tobst: Observed time
    - reflevelt: Reference level of bacteria
    - activethrest: Threshold for activation

    Returns:
    - List of derivatives [dbstandard_cc, dpstandard_cc] representing the rate of change of bacteria and phage populations.
    """
    bstandard_cc, pstandard_cc = populations_standard_cc  # Unpack the populations_standard_cc list into separate variables bstandard_cc and pstandard_cc
    theta_g_f = theta_p_f = theta_b_f = theta_d_f = theta_c_f = 1  # Initialize the theta parameters to 1
    dbstandard_cc = ((theta_g_f * rt * bstandard_cc) - (theta_p_f * at * bstandard_cc * pstandard_cc) - ((theta_c_f * rt * bstandard_cc**2) / (kt)))  # Calculate the derivative of bacteria population using the Lotka-Volterra equation
    dpstandard_cc = ((theta_b_f * ct * at * bstandard_cc * pstandard_cc) - (theta_d_f * mt * pstandard_cc))  # Calculate the derivative of phage population using the Lotka-Volterra equation
    return [dbstandard_cc, dpstandard_cc]  # Return the list of derivatives [dbstandard_cc, dpstandard_cc]

# Function to display carrying capacity as a weight by itself
def plot_carry_capacity(time_final, carry_capacity, thresholdlevel, reference_level):
    """
    Function to plot the carrying capacity as a weight by itself.

    Parameters:
    - time_final: The final time value
    - carry_capacity: List of carrying capacity values
    - thresholdlevel: The threshold level for activation
    - reference_level: The reference level of bacteria

    Returns:
    - None
    """
    # Create time array
    t = np.linspace(0, time_final, len(carry_capacity))

    # Set up the figure and subplot
    fig, ax = plt.subplots(figsize=(3.632, 0.456), dpi=250)  # 230x29 pts

    # Define plot settings based on reference_level
    if reference_level == 1:
        y_lim = (1e-4, 1e4)
        y_ticks = [1e-3, 1, 1e3]
    elif reference_level == 0.10:
        y_lim = (1e-5, 1e5)
        y_ticks = [1e-4, 1, 1e4]
    else:
        raise ValueError("Unsupported reference_level")

    # Plot data
    ax.semilogy(t, carry_capacity, color='b', linewidth=1.50, label='Carry Capacity')

    # Add threshold line
    ax.axhline(y=thresholdlevel, color='tab:grey', linestyle='dotted', linewidth=0.75)

    # Add grey shading from 10^-4 to 1
    ax.axhspan(1e-4, 1, facecolor='tab:grey', alpha=0.3)

    # Set y-axis limits and ticks
    ax.set_ylim(y_lim)
    ax.set_yticks(y_ticks)

    # Format y-axis labels as 10^x
    def y_fmt(y, pos):
        if y == 1:
            return '1'
        decades = int(np.log10(y))
        return r'$10^{%d}$' % decades

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))

    # Adjust tick parameters
    ax.tick_params(axis='both', which='major', labelsize=6, pad=2, width=0.9)
    ax.tick_params(axis='both', which='minor', width=0.6)

    # Set x-axis scale and ticks based on time_final
    if time_final > 1000:
        ax.set_xscale('log')
        x_ticks = [10**i for i in range(int(np.log10(time_final))+1)]
        ax.set_xticks(x_ticks)
        ax.set_xticklabels([f'$10^{int(np.log10(x))}$' if x > 1 else '1' for x in x_ticks])
    else:
        ax.set_xscale('linear')
        num_ticks = 6
        x_ticks = np.linspace(0, time_final, num_ticks)
        ax.set_xticks(x_ticks)
        ax.set_xticklabels([f'{x:.0f}' for x in x_ticks])

    # Set x-axis label
    ax.set_xlabel('Time (h)', fontsize=7, labelpad=2)

    # Set y-axis label on the right side
    ax.yaxis.set_label_position("right")
    ax.set_ylabel('Carrying Capacity', fontsize=5, labelpad=2)

    # Adjust layout
    plt.tight_layout()

    # Add a thin border around the entire figure
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(0.75)

    # Save and show the figure
    plt.savefig('Carry_Capacity.svg', dpi=250, bbox_inches='tight')
    plt.show()
# New population dynamic displaying function that is used on the carrying capacity

def population_plot_comparison_and_tipping_points_cc(time_list, time_final, tipping_point_list, tipping_points_of_weight_predation, tipping_points_of_weights_burst, tipping_points_of_weights_carrying_capacity, Agent_full_phage, Agent_full_bacteria, Agent_adaptive_phage, Agent_adaptive_bacteria):
    print('Below are the graphs representing both the tipping points and populations plot of both models')
    """
    Function to plot the population comparison and tipping points of the carrying capacity model.

    Parameters:
    - time_list: List of time values
    - time_final: The final time value
    - tipping_point_list: List of tipping points
    - tipping_points_of_weight_predation: List of tipping points related to predation weight
    - tipping_points_of_weights_burst: List of tipping points related to burst weight
    - tipping_points_of_weights_carrying_capacity: List of tipping points related to carrying capacity weight
    - Agent_full_phage: List of phage populations in the full model
    - Agent_full_bacteria: List of bacteria populations in the full model
    - Agent_adaptive_phage: List of phage populations in the adaptive model
    - Agent_adaptive_bacteria: List of bacteria populations in the adaptive model

    Returns:
    - None
    """
    # Calculate the figure size to maintain the 227:127 ratio
    width_inches = 3.632
    height_inches = width_inches * (114 / 215)

    # Set up the figure and subplots
    fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=250)

    ax.semilogy(time_list, Agent_full_bacteria, label='Bacteria', color='b', linestyle='solid', linewidth=0.75)
    ax.semilogy(time_list, Agent_full_phage, label='Phage', color='r', linestyle='solid', linewidth=0.75)
    ax.semilogy(time_list, Agent_adaptive_bacteria, label='Adaptive', color='k', linestyle='dashed', dashes=(5,5), linewidth=0.75)
    ax.semilogy(time_list, Agent_adaptive_phage, color='k', linestyle='dashed', dashes=(5,5), linewidth=0.75)

    ax.set_xlabel("Time (h)", fontsize=7, labelpad=2)
    ax.set_ylabel('Concentration ($ml^{-1}$)', fontsize=7, labelpad=2)

    # Set x-axis scale based on time_final
    if time_final > 1000:
        ax.set_xscale('log')
        ax.set_xlim(1, time_final)  # Start from 1 to avoid log(0) issues

        # Add minor ticks for log scale (x-axis only)
        locmin = ticker.LogLocator(base=10.0, subs=(0.2,0.4,0.6,0.8), numticks=12)
        ax.xaxis.set_minor_locator(locmin)
        ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    else:
        ax.set_xscale('linear')
        ax.set_xlim(0, time_final)

        # Add minor ticks for linear scale (x-axis only)
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())

    # Remove minor ticks from y-axis
    ax.yaxis.set_minor_locator(ticker.NullLocator())

    if round(np.log10(max(Agent_full_phage))) > round(np.log10(max(Agent_full_bacteria))):
        max_limit = (1 * (10**round(np.log10(max(Agent_full_phage)))))
        ax.set_ylim(1, max_limit)
    else:
        max_limit = (1 * (10**round(np.log10(max(Agent_full_bacteria)))))
        ax.set_ylim(1, max_limit)

    if max_limit < 10**9:
        ax.set_yticks([10**0, 10**3, 10**6, 10**9])
    else:
        ax.set_yticks([10**0, 10**3, 10**6, 10**9, 10**12])

    # Format y-axis labels as 10^x
    def y_fmt(y, pos):
        decades = int(np.log10(y))
        return r'$10^{%d}$' % decades

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_fmt))

    # Adjust x-axis ticks
    if time_final > 1000:
        x_ticks = [10**i for i in range(int(np.log10(time_final))+1)]
        ax.set_xticks(x_ticks)
        ax.set_xticklabels([f'$10^{int(np.log10(x))}$' if x > 1 else '1' for x in x_ticks])
    else:
        num_ticks = 6
        x_ticks = np.linspace(0, time_final, num_ticks)
        ax.set_xticks(x_ticks)
        ax.set_xticklabels([f'{x:.0f}' for x in x_ticks])

    # Setting up the vertical lines for tipping points
    if len(tipping_point_list) != 0:
        for tps in tipping_point_list:
            if tps in tipping_points_of_weight_predation:
                ax.vlines(x=tps, ymin=0, ymax=(float(Agent_full_phage[np.where(time_list == tps)[0][0]])), color='tab:grey', linestyle='solid', linewidth=0.75)
            elif tps in tipping_points_of_weights_burst:
                ax.vlines(x=tps, ymin=0, ymax=(float(Agent_full_bacteria[np.where(time_list == tps)[0][0]])), color='tab:grey', linestyle='solid', linewidth=0.75)
            elif tps in tipping_points_of_weights_carrying_capacity:
                ax.vlines(x=tps, ymin=0, ymax=(float(Agent_full_bacteria[np.where(time_list == tps)[0][0]])), color='tab:grey', linestyle='solid', linewidth=0.75)
            else:
                print('error')

    ax.legend(fontsize=5, loc='best')

    # Adjust tick parameters
    ax.tick_params(axis='both', which='major', labelsize=6, pad=2)
    ax.tick_params(axis='x', which='minor', labelsize=4, pad=2)

    plt.tight_layout(pad=0.1)

    # Add a thin border around the entire figure
    fig.patch.set_linewidth(0.5)
    fig.patch.set_edgecolor('black')

    # Save and show the figure
    plt.savefig('population_comparison_graph.svg', dpi=250, bbox_inches='tight')
    plt.show()
    return

def plot_combined_graphs(time_final, threshholdlevel, reference_level, weight_growth, weight_predation, weight_burst, weight_decay, carry_capacity, tipping_point_list_predation, tipping_point_list_burst):
    """
    Plot the combined graphs of various weights and carrying capacity.

    Parameters:
    - time_final (float): The final time of the simulation.
    - threshholdlevel (float): The threshold level.
    - reference_level (float): The reference level.
    - weight_growth (array-like): The growth weight data.
    - weight_predation (array-like): The predation weight data.
    - weight_burst (array-like): The burst weight data.
    - weight_decay (array-like): The decay weight data.
    - carry_capacity (array-like): The carrying capacity data.
    - tipping_point_list_predation (list): The list of tipping points for predation.
    - tipping_point_list_burst (list): The list of tipping points for burst.

    Returns:
    None
    """
    # Create time array
    t = np.linspace(0, time_final, len(weight_growth))

    # Set up the figure and subplots
    fig, axs = plt.subplots(5, 1, sharex=True, figsize=(3.632, 2.28), dpi=250)

    # Define plot settings based on reference_level
    if reference_level == 1:
        y_lim = (1e-4, 1e4)
        y_ticks = [1e-3, 1, 1e3]
    elif reference_level == 0.10:
        y_lim = (1e-5, 1e5)
        y_ticks = [1e-4, 1, 1e4]
    else:
        raise ValueError("Unsupported reference_level")

    # Plot data
    plot_data = [
        (weight_growth, 'Growth Weight', 'Growth', 'b'),
        (weight_predation, 'Predation Weight', 'Predation', 'b'),
        (carry_capacity, 'Carrying Capacity', 'Carrying Capacity', 'b'),
        (weight_burst, 'Burst Weight', 'Burst', 'r'),
        (weight_decay, 'Decay Weight', 'Decay', 'r')
    ]

    for i, (data, label, ylabel, color) in enumerate(plot_data):
        ax = axs[i]
        ax.semilogy(t, data, label=label, color=color, linewidth=1.50)
        ax.yaxis.set_label_position("right")
        ax.set_ylabel(ylabel, fontsize=5, labelpad=2)
        ax.set_ylim(y_lim)
        ax.set_yticks(y_ticks)
        #ax.axhline(y=threshholdlevel, color='tab:grey', linestyle='dotted', linewidth=0.75)
        ax.axhline(y=0, color='k', linewidth=0.75)
        ax.axhspan(1e-5, 1e0, facecolor='tab:grey', alpha=0.3)

        # Format y-axis labels as 10^x
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '1' if y == 1 else r'$10^{%d}$' % int(np.log10(y))))

        # Adjust tick parameters
        ax.tick_params(axis='both', which='major', labelsize=6, pad=2, width=0.9)
        ax.tick_params(axis='both', which='minor', width=0.6)
        if i < 4:  # Remove x-axis labels for all but the last subplot
            ax.tick_params(axis='x', labelbottom=False)
        if i == 1:
            for tps in tipping_point_list_predation:
                ax.vlines(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)
        if i == 2:
            for tps in tipping_point_list_burst:
                ax.vlines(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)

        # Keep all spines visible
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_linewidth(1.15)

    # Set x-label for the bottom subplot
    axs[-1].set_xlabel('Time (h)', fontsize=7, labelpad=2)

    # Set x-axis scale based on time_final
    if time_final > 1000:
        for ax in axs:
            ax.set_xscale('log')
        x_ticks = [10**i for i in range(int(np.log10(time_final))+1)]
        axs[-1].set_xticks(x_ticks)
        axs[-1].set_xticklabels([f'$10^{int(np.log10(x))}$' if x > 1 else '1' for x in x_ticks])
        axs[-1].set_xlim(1, time_final)  # Start from 1 to avoid log(0) issues
    else:
        for ax in axs:
            ax.set_xscale('linear')
        num_ticks = 6
        x_ticks = np.linspace(0, time_final, num_ticks)
        axs[-1].set_xticks(x_ticks)
        axs[-1].set_xticklabels([f'{x:.0f}' for x in x_ticks])
        axs[-1].set_xlim(0, time_final)

    # Adjust layout to remove space between subplots
    plt.subplots_adjust(hspace=0, left=0.15)  # Increased left margin for the new label

    # Add a thin border around the entire figure
    fig.patch.set_linewidth(.5)
    fig.patch.set_edgecolor('black')

    # Add the larger "Weights" label on the left side
    fig.text(0.02, 0.5, 'Weights', va='center', rotation='vertical', fontsize=10)

    # Save and show the figure
    plt.savefig('Combined_Weights_and_Capacity_cc.svg', dpi=250, bbox_inches='tight')
    plt.savefig('Combined_Weights_and_Capacity_cc.png', dpi=250, bbox_inches='tight')
    plt.savefig('Combined_Weights_and_Capacity_cc.pdf', dpi=250, bbox_inches='tight')
    plt.show()
    return

def relative_error_graphs_cc(time_list, time_final, p_bac_relative_error_percent, p_pha_relative_error_percent, p_com_relative_error_percent, tipping_point_list_adaptive, tipping_point_list_full):
    """
    Plot the relative error graphs for bacteria, phage, and combined error.

    Parameters:
    - time_list (array-like): List of time points.
    - time_final (float): The final time of the simulation.
    - p_bac_relative_error_percent (array-like): Relative error of bacteria.
    - p_pha_relative_error_percent (array-like): Relative error of phage.
    - p_com_relative_error_percent (array-like): Combined relative error.
    - tipping_point_list_adaptive (list): List of tipping points for the adaptive model.
    - tipping_point_list_full (list): List of tipping points for the full model.

    Returns:
    - p_com_relative_error_percent (array-like): Combined relative error.
    """
    for t in range(len(p_bac_relative_error_percent)):
        p_com_relative_error_percent_ind = np.mean([p_bac_relative_error_percent[t], p_pha_relative_error_percent[t]])
        p_com_relative_error_percent.append(p_com_relative_error_percent_ind)

    # Calculate the figure size to maintain the 227:127 ratio
    width_inches = 3.632
    height_inches = width_inches * (114 / 215)

    # Set up the figure and subplots
    fig, ax = plt.subplots(figsize=(width_inches, height_inches), dpi=250)

    ax.plot(time_list, p_bac_relative_error_percent, label='Relative Error Bacteria', color='b', linewidth=0.75)
    ax.plot(time_list, p_pha_relative_error_percent, label='Relative Error Phage', color='r', linewidth=0.75)
    ax.plot(time_list, p_com_relative_error_percent, label="Relative Error", color='#4f5050', linewidth=0.8)

    ax.set_xlabel("Time (h)", fontsize=7, labelpad=2)
    ax.set_ylabel("Relative Error (%)", fontsize=7, labelpad=2)
    ax.set_xlim(1, time_final)  # Start from 1 to avoid log(0) issues

    for tps in tipping_point_list_adaptive:
        if tps == tipping_point_list_adaptive[0]:
            ax.axvline(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75, label='Adaptive Model Tipping Point')
        else:
            ax.axvline(x=tps, ymin=0, ymax=1, color='tab:grey', linestyle='solid', linewidth=0.75)
    for tps in tipping_point_list_full:
        if tps == tipping_point_list_full[0]:
            ax.axvline(x=tps, ymin=0, ymax=1, color='k', linestyle=(0, (3, 6, 1, 6)), linewidth=1, label='Full Model Tipping Point')
        else:
            ax.axvline(x=tps, ymin=0, ymax=1, color='k', linestyle=(0, (3, 6, 1, 6)), linewidth=1)

    ax.legend(fontsize=6, loc='best')

    # Adjust tick parameters
    ax.tick_params(axis='both', which='major', labelsize=6, pad=2)
    ax.tick_params(axis='x', which='minor', labelsize=4, pad=2)

    plt.tight_layout(pad=0.1)

    # Add a thin border around the entire figure
    fig.patch.set_linewidth(0.5)
    fig.patch.set_edgecolor('black')

    # Calculate the maximum relative error
    max_relative_error = max(max(p_bac_relative_error_percent), max(p_pha_relative_error_percent))

    # Set up the main plot
    ax.set_ylim(0, 15)
    ax.set_yticks([0, 5, 10, 15])
    ax.set_yticklabels(['0', '5', '10', '15'])

    # Create an inlay if the maximum relative error exceeds 15%
    if max_relative_error > 15:
        # Find the last tipping point
        last_tipping_point = max(max(tipping_point_list_adaptive or [0]), max(tipping_point_list_full or [0]))

        # Calculate inlay position (between last tipping point and end of graph)
        inlay_left = 0.5 + (0.45 * np.log10(last_tipping_point) / np.log10(time_final))
        inlay_width = 0.45 * (1 - np.log10(last_tipping_point) / np.log10(time_final))

        # Create an inlay axes
        inlay_ax = fig.add_axes([inlay_left, 0.65, inlay_width, 0.25])  # [left, bottom, width, height]

        # Plot the full range data in the inlay
        inlay_ax.semilogx(time_list, p_bac_relative_error_percent, color='b', linewidth=0.5)
        inlay_ax.semilogx(time_list, p_pha_relative_error_percent, color='r', linewidth=0.5)
        inlay_ax.semilogx(time_list, p_com_relative_error_percent, color='#4f5050', linewidth=0.5)

        # Set the y-axis limits for the inlay
        inlay_ax.set_ylim(0, max(50, max_relative_error * 1.1))  # Set upper limit to at least 50 or 10% above max error

        # Set x-axis limits for the inlay
        inlay_ax.set_xlim(1, time_final)

        # Set x-ticks for the inlay (similar to main plot)
        inlay_ax.set_xticks(x_ticks)
        inlay_ax.set_xticklabels([f'$10^{int(np.log10(x))}$' if x > 1 else '1' for x in x_ticks], fontsize=5)

        # Set y-ticks for inlay
        inlay_ax.set_yticks([0, 25, 50])
        inlay_ax.set_yticklabels(['0', '25', '50'])

        # Adjust tick parameters for the inlay
        inlay_ax.tick_params(axis='both', which='major', labelsize=5)

    # Save the figure with inlay (if applicable)
    plt.savefig('relative_error_with_inlay_cc.svg', dpi=250, bbox_inches='tight')
    plt.show()
    return [p_com_relative_error_percent]


## Storage lists utilized in carrying capacity included model

In [ ]:
# Weight lists for the adaptive model
wg_cc = wp_cc = wb_cc = wd_cc = wk_cc = []
# Weight lists for the full model
wg_full_cc = wp_full_cc = wb_full_cc = wd_full_cc = wk_full_cc = []
# Tipping point lists for adaptive model
tp_g_cc = tp_p_cc = tp_b_cc = tp_d_cc = tp_k_cc =[]
# Tipping point lists for full model
tp_g_full_cc = tp_p_full_cc = tp_b_full_cc = tp_d_full_cc = tp_k_full_cc = []
#error lists
p_error_cc = p_rel_error_cc = p_rel_error_percent_cc = b_error_cc = b_rel_error_cc = b_rel_error_percent_cc = p_com_relative_error_percent_cc = []

## ODE solver and solutions for Lotka-Volterra model with carrying capacity

In [ ]:
# Initial populations
y0 = [(bi_cc),(pi_cc)]

# Time span for the ODE solver
timespan_cc = (0,tobs_cc)

# Time points for the ODE solver
t_for_solver_cc = np.linspace(0, tobs_cc, 10000)

# ODE solvers for the carrying capacity using the theta approach model
sol_adapt_cc_module = solve_ivp(theta_approach_model_carrying_capacity, timespan_cc, y0=y0, args=para_test_cc, method='LSODA', t_eval=t_for_solver_cc, rtol=10**-13, atol=10**-16)

# ODE solvers for the carrying capacity using the full model
sol_full_cc_module = solve_ivp(lotka_volterra_standard_cc, timespan_cc, y0=y0, args=para_test_cc, method='LSODA', t_eval=t_for_solver_cc, rtol=10**-13, atol=10**-16)

# Time points for the adaptive model
t_cc = sol_adapt_cc_module.t

# Time points for the full model
t_full_cc = sol_full_cc_module.t

# Bacteria populations for the adaptive model
b_cc = sol_adapt_cc_module.y[0, :]

# Phage populations for the adaptive model
p_cc = sol_adapt_cc_module.y[1, :]

# Bacteria populations for the full model
b_full_cc = sol_full_cc_module.y[0, :]

# Phage populations for the full model
p_full_cc = sol_full_cc_module.y[1, :]

## Calculation of weights per processes, tipping points and errors for adaptive and full model
Running this code generates weights and tipping points for both models, and errors analysis (absolute error, and relative error) between the two models.

In [ ]:
# weight definitions for the carrying capacity models
wg_function_cc = para_test_cc[5] * (para_test_cc[0] * b_cc/(para_test_cc[6] * b_cc))
wp_function_cc = para_test_cc[5] * ((para_test_cc[1] * p_cc * b_cc)/(para_test_cc[6] * b_cc))
wb_function_cc = para_test_cc[5] * ((para_test_cc[2] * para_test_cc[1] * p_cc * b_cc)/(para_test_cc[6] * p_cc))
wd_function_cc = para_test_cc[5] * ((para_test_cc[3] * p_cc)/(para_test_cc[6] * p_cc))
wk_function_cc = para_test_cc[5] * ((para_test_cc[0] * b_cc**2)/(para_test_cc[6] * b_cc * para_test_cc[4]))
##Weight_functions for ful model
wg_function_full_cc = para_test_cc[5] * (para_test_cc[0] * b_full_cc/(para_test_cc[6] * b_full_cc))
wp_function_full_cc = para_test_cc[5] * ((para_test_cc[1] * p_full_cc * b_full_cc)/(para_test_cc[6] * b_full_cc))
wb_function_full_cc = para_test_cc[5] * ((para_test_cc[2] * para_test_cc[1] * p_full_cc * b_full_cc)/(para_test_cc[6] * p_full_cc))
wd_function_full_cc = para_test_cc[5] * ((para_test_cc[3] * p_full_cc)/(para_test_cc[6] * p_full_cc))
wk_function_full_cc = para_test_cc[5] * ((para_test_cc[0] * b_full_cc**2)/(para_test_cc[6] * b_full_cc * para_test_cc[4]))
#Finding the weights for each adaptive processes
wg_cc = weights_finder(wg_function_cc,t_cc,wg_cc)
wp_cc = weights_finder(wp_function_cc,t_cc,wp_cc)
wb_cc = weights_finder(wb_function_cc,t_cc,wb_cc)
wd_cc = weights_finder(wd_function_cc,t_cc,wd_cc)
wk_cc = weights_finder(wk_function_cc,t_cc,wk_cc)
#Finding the weights for each full processes
wg_full_cc = weights_finder(wg_function_full_cc,t_cc,wg_full_cc)
wp_full_cc = weights_finder(wp_function_full_cc,t_cc,wp_full_cc)
wd_full_cc = weights_finder(wd_function_full_cc,t_cc,wd_full_cc)
wb_full_cc = weights_finder(wb_function_full_cc,t_cc,wb_full_cc)
wk_full_cc = weights_finder(wk_function_full_cc,t_cc,wk_full_cc)
#Finding the tipping points for each adaptive process, if there is one
tp_g_cc = tipping_point_finder(wg_cc,t_cc,tp_g_cc,para_test_cc[7])
tp_p_cc = tipping_point_finder(wp_cc,t_cc,tp_p_cc,para_test_cc[7])
tp_b_cc = tipping_point_finder(wb_cc,t_cc,tp_b_cc,para_test_cc[7])
tp_d_cc = tipping_point_finder(wd_cc,t_cc,tp_d_cc,para_test_cc[7])
tp_k_cc = tipping_point_finder(wk_cc,t_cc,tp_k_cc,para_test_cc[7])
tp_cc = tp_g_cc + tp_p_cc + tp_b_cc + tp_d_cc + tp_k_cc
#Finding the tipping points for each full process, if there is one
tp_g_full_cc = tipping_point_finder(wg_full_cc,t_full_cc,tp_g_full_cc,para_test_cc[7])
tp_p_full_cc = tipping_point_finder(wp_full_cc,t_full_cc,tp_p_full_cc,para_test_cc[7])
tp_b_full_cc = tipping_point_finder(wb_full_cc,t_full_cc,tp_b_full_cc,para_test_cc[7])
tp_d_full_cc = tipping_point_finder(wd_full_cc,t_full_cc,tp_b_full_cc,para_test_cc[7])
tp_k_full_cc = tipping_point_finder(wk_full_cc,t_full_cc,tp_k_full_cc,para_test_cc[7])
tp_full_cc = tp_g_full_cc + tp_p_full_cc + tp_b_full_cc + tp_d_full_cc + tp_k_full_cc
# Error calculations
p_error_cc = true_error(t_cc,p_full_cc,p_cc,p_error_cc)
p_rel_error_cc = relative_error(t_cc,p_error_cc,p_full_cc,p_rel_error_cc)
p_rel_error_percent_cc = relative_error_percentage_finder(p_rel_error_cc,t_cc,p_rel_error_percent_cc)
b_error_cc = true_error(t_cc,b_full_cc,b_cc,b_error_cc)
b_rel_error_cc = relative_error(t_cc,b_error_cc,b_full_cc,b_rel_error_cc)
b_rel_error_percent_cc = relative_error_percentage_finder(b_rel_error_cc,t_cc,b_rel_error_percent_cc)

## Weight subplots of the carrying capacity and other processes in the models utilizing carrying capacity

In [ ]:
plot_combined_graphs(tobs_cc,thresholdlevel_cc,reference_level_cc,wg_full_cc,wp_full_cc,wb_full_cc,wd_full_cc,wk_full_cc,tp_p_full_cc,tp_b_full_cc)

## Population dynamics of the carrying capacity containing models

In [ ]:
population_plot_comparison_and_tipping_points_cc(t_cc,tobs_cc,tp_full_cc,tp_p_full_cc,tp_b_full_cc,tp_k_full_cc,p_full_cc,b_full_cc,p_cc,b_cc)

## Relative error vs time graph for both agents in scenario with carrying capacity
Running this code generates a relative error vs time graph for both population as well as an average error of the scenario.

In [ ]:
relative_error_graphs_cc(t_cc,tobs,b_rel_error_percent_cc,p_rel_error_percent_cc,p_com_relative_error_percent_cc,tp_cc, tp_full_cc)

# Storaging outputs in carrying capacity added model
## Storage of arrays related to time-steps
Running this code places all arrays and lists into a singular .xslx sheet titled data_output.xslx

In [ ]:
df_for_xslx = pd.DataFrame({
    'time_full': t_full_cc,
    'time': t_cc,
    'phage_full': p_full_cc,
    'bacteria_full': b_full_cc,
    'phage_adaptive': p_cc,
    'bacteria_adaptive': b_cc,
    'weight_growth_full': wg_full_cc,
    'weight_predation_full': wp_full_cc,
    'weight_burst_full': wb_full_cc,
    'weight_decay_full': wd_full_cc,
    'weight_carrying_capacity_full': wk_full_cc,
    'weight_growth': wg_cc,
    'weight_predation': wp_cc,
    'weight_burst': wb_cc,
    'weight_decay': wd_cc,
    'weight_carrying_capacity': wk_cc,
    'error_phage': p_error_cc,
    'error_bacteria': b_error_cc,
    'rel_error_phage': p_rel_error_cc,
    'rel_error_bacteria': b_rel_error_cc,
    'rel_error_phage_per': p_rel_error_percent_cc,
    'rel_error_bac_per': b_rel_error_percent_cc,
    'rel_error_com_per': p_com_relative_error_percent_cc,
     })
excel_file = 'data_output_cc.xlsx'
df_for_xslx.to_excel(excel_file, index=False, sheet_name='Sheet1')